In [2]:
# !git add .
# !git commit -m "ran hueristic features on euro dataset and updates RSS collection"
# !git pull
# !git push -u origin main

In [5]:
# !pip install "numpy<2"

## Sports Misinformation Classification Tool

In [7]:
import mysql.connector
from sqlalchemy import create_engine
import pandas as pd

### Connect to the Database

In [18]:
username = 'root'  
password = 'password'
host = 'localhost' 
database = 'sports_news_db'

connection = mysql.connector.connect(
    host=host,
    user=username,
    password=password,
    database=database
)

if connection.is_connected():
    print("Connected to the database successfully!")

engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{host}/{database}", echo=False)

Connected to the database successfully!


#### Use this format to insert into the database

INSERT INTO articles (team_or_player, source, publication_date, content, trust_score, classification, link)
VALUES
('New York Yankees, Los Angeles Lakers', 'ESPN', '2024-10-27', 'Yankees article content example.', 85.00, 'real', 'https://example.com/article1'),
('Los Angeles Lakers', 'Twitter', '2024-10-27', 'Lakers article content example.', 60.00, 'fake', 'https://example.com/article2');


#### Table fields 
Table Name: articles

Fields: 

     id INT AUTO_INCREMENT PRIMARY KEY,
     
     team_or_player VARCHAR(500), (This will be the article title that we can query- usually includes teams or names in it)
     
     source VARCHAR(200),
     
     publication_date DATE,
     
     content TEXT,
     
     trust_score DECIMAL(5, 2), 
     
     classification ENUM('credible', 'uncredible', 'unknown') DEFAULT 'unknown',

     link VARCHAR(255)

### Simulate Tool Working

In [118]:
team_or_player = input("Enter the team or player's name: ")

query = f"SELECT * FROM articles WHERE team_or_player LIKE '%{team_or_player}%'" #search for entered name/team in the title 
df_result = pd.read_sql(query, con=engine, params={'team_or_player': team_or_player})

#get results
if not df_result.empty:
    print(f"Articles related to {team_or_player}:")
    display(df_result)
else:
    print(f"No articles found for {team_or_player}.")

Enter the team or player's name:  x


No articles found for x.


### Get Article Entries from RSS Feeds

In [5]:
# # reddit API - collected 1876 posts
# import csv
# import praw
# from datetime import datetime


# reddit = praw.Reddit(
#     client_id='w-kwRyPigyjYeG9DOiDc8g', 
#     client_secret='ZeDsvNH2YlpVH7F9wEWPkt5wkjLzqA',  
#     user_agent='sports_misinfo_script'  
# )

# subreddit = reddit.subreddit('sports+fantasyfootball') #2 subreddits 

# recent_posts = []
# for post in subreddit.new(limit= 5000):
#     created_date = datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S')
#     recent_posts.append({
#         'title': post.title,
#         'score': post.score,
#         'url': post.url,
#         'id': post.id,
#         'author': str(post.author),
#         'text': post.selftext,
#         'created_date': created_date,
#         'num_comments': post.num_comments,
#         'subreddit': post.subreddit.display_name,

#     })

# print(f"Fetched {len(recent_posts)} posts")

# for i, post in enumerate(recent_posts[:5]):
#     print(f"{i+1}. Title: {post['title']} | Score: {post['score']} | URL: {post['url']}")


# #-------------------------------
# #Save the data to a csv file
# fieldnames = ['title', 'score', 'url', 'id', 'author', 'text', 'created_date', 'num_comments', 'subreddit']

# with open('recent_sports_reddit_posts.csv', mode='w', newline='', encoding='utf-8') as csv_file:
#     writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

#     writer.writeheader()

#     for post in recent_posts:
#         writer.writerow(post)

# print("Data successfully saved to 'recent_sports_reddit_posts.csv'")
# print("Data saved")


In [15]:
import feedparser
import requests
from bs4 import BeautifulSoup

#add urls to this list to parse
url_list = [
        #MBFC High Credibility
        "https://www.espn.com/espn/rss/news", #ESPN top headlines
        "https://deadspin.com/rss/", #Deadspin 
        "https://feeds.abcnews.com/abcnews/sportsheadlines" #ABC news- espn sports
        "https://rss.nytimes.com/services/xml/rss/nyt/Sports.xml", #NYT
        "https://www.nytimes.com/athletic/rss/news/"#The Athletic- acquired by NYT
        "https://www.mlb.com/feeds/news/rss.xml", #MLB news 
        "https://www.reutersagency.com/feed/?best-topics=sports&post_type=best", # Reuters
        "https://sports.yahoo.com/rss/", #Yahoo News
        "https://www.cbssports.com/rss/headlines/" #CBS sports general headlines 

        #MBFC questionable sources or medium credibility
        "https://notthebee.com/feed", #not the bee
        "https://uproxx.com/sports/feed/", #uproxx   
        "https://www.vibe.com/c/news/sports/feed/", #The vibe - medium cred 
        "https://moxie.foxnews.com/google-publisher/sports.xml", #fox news 
        "https://nypost.com/sports/feed/" #NY post - medium cred

        #NO MBFC RATING
        "https://www.sportscollectorsdaily.com/feed/", #Sports Collectors Daily 
        "https://news.sportslogos.net/feed/", #SportsLogos.Net
        "https://www.sportingnews.com/us/rss", #sportingnews.com
        "https://www.sportskeeda.com/feed",#sportskeeda.com
        "https://sportsweez.com/feed/", #sportsweez
        "https://sportsbrackets.net/feed/", #sportsbracket
        "https://21sports.com/feed/", # 21 sports
        "https://www.essentiallysports.com/feed/", #essentiallysports
        "https://boxingnewsonline.net/feed/", #boxing news
        "https://www.thecoldwire.com/feed/", #the cold wire
        "https://sportsdark.com/feed/", #sports dark
]
    
entries = [] #list of dictionaries

#--------------DEFINE FUNCTION TO SCRAPE-------------------
def scrape_article_content(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")

        #extract content from common tags
        article_body = (
            soup.find("article") or
            soup.find("div", {"class": "post-content"}) or
            soup.find("div", class_="article-body") or
            soup.find("div", class_="article-content") or
            soup.find("section", class_="article-section") or
            soup.find("div", class_="main-content") or
            soup.find("div", class_="content-body")
        )
        
        if article_body:
            paragraphs = article_body.find_all("p")
        else:
            paragraphs = soup.find_all("p")

        #join all paragraphs into a single string
        article_content = " ".join(p.get_text() for p in paragraphs)
        return article_content.strip() if article_content else "No content found"

    except Exception as e:
        print(f"Failed to scrape content from {url}: {e}")
        return "Failed to fetch content"


#run the function to collect feeds and scrape
for url in url_list:
    feed = feedparser.parse(url)
    #feed_title= feed.feed.title
    feed_title = getattr(feed.feed, "title", None)

    
    for entry in feed.entries:
        entry_title = getattr(entry, "title", None)
        entry_link = getattr(entry, "link", None)
        entry_published_date = getattr(entry, "published", None)
        entry_summary = getattr(entry, "summary", None)
        entry_content = scrape_article_content(entry_link) #scrape
        
        entries.append({
            "feed_title": feed_title,
            "entry_title": entry_title,
            "entry_link": entry_link,
            "entry_published_date": entry_published_date,
            "entry_summary": entry_summary,
            "entry_content": entry_content,
        })

df = pd.DataFrame(entries)
#print(df)

df.to_csv('RSS_sports_feeds_11-17.csv', index=False)

Failed to scrape content from https://www.espn.com/college-football/story/_/id/42447741/oregon-remains-unanimous-no-1-ap-top-25-notre-dame-6: 403 Client Error: Forbidden for url: https://www.espn.com/college-football/story/_/id/42447741/oregon-remains-unanimous-no-1-ap-top-25-notre-dame-6
Failed to scrape content from https://www.espn.com/nba/story/_/id/42447996/hornets-lamelo-ball-fined-100k-using-anti-gay-term: 403 Client Error: Forbidden for url: https://www.espn.com/nba/story/_/id/42447996/hornets-lamelo-ball-fined-100k-using-anti-gay-term
Failed to scrape content from https://www.espn.com/college-football/story/_/id/42447431/sources-top-2025-qb-julian-lewis-decommits-usc: 403 Client Error: Forbidden for url: https://www.espn.com/college-football/story/_/id/42447431/sources-top-2025-qb-julian-lewis-decommits-usc
Failed to scrape content from https://www.espn.com/nba/story/_/id/42449490/celtics-celebrate-nba-title-white-house-thursday: 403 Client Error: Forbidden for url: https://ww

### Attempt to Scrape Links without RSS Feeds- not very effective 

In [9]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd

# uncredible_urls = [
#     "https://www.tellerreport.com/sports",
#     "https://www.newsbreak.com/mountain-view-ca-sports",
#     "https://newsrnd.com/sports",
#     "https://baltimorecitywire.com/stories/tag/53-sports"
# ]

# # Function to scrape article details
# def scrape_article(url):
#     try:
#         response = requests.get(url, timeout=10)
#         response.raise_for_status()
#         soup = BeautifulSoup(response.content, "html.parser")

#         # Extract the article title
#         title = soup.find("h1").get_text() if soup.find("h1") else soup.title.get_text()

#         # Extract the publication date (common in <time> or meta tags)
#         date = soup.find("time")
#         if date:
#             publication_date = date.get("datetime") or date.get_text()
#         else:
#             date_meta = soup.find("meta", {"name": "article:published_time"})
#             publication_date = date_meta["content"] if date_meta else "No date found"

#         # Extract the article content
#         content_container = (
#             soup.find("article") or
#             soup.find("div", class_=["post-content", "article-body", "article-content", "content-body"])
#         )
#         if content_container:
#             paragraphs = content_container.find_all("p")
#         else:
#             paragraphs = soup.find_all("p")

#         content = "\n".join(p.get_text() for p in paragraphs)

#         return {
#             "Title": title.strip(),
#             "Publication Date": publication_date.strip(),
#             "Content": content.strip()[:500] + "...",
#             "Link": url
#         }

#     except Exception as e:
#         print(f"Failed to scrape {url}: {e}")
#         return None

# articles = []
# for url in uncredible_urls:
#     print(f"Scraping: {url}")
#     article_details = scrape_article(url)
#     if article_details:
#         articles.append(article_details)

# for article in articles:
#     print("\n--- Article ---")
#     print(f"Title: {article['Title']}")
#     print(f"Date: {article['Publication Date']}")
#     print(f"Content Preview: {article['Content']}")
#     print(f"Link: {article['Link']}\n")

# df = pd.DataFrame(articles)
# output_file = "scraped_uncredible_articles.csv"
# df.to_csv(output_file, index=False)
# print(f"\nResults saved to {output_file}")

### Format DF to match DB table

In [123]:
from datetime import datetime
from dateutil import parser

df['publication_date'] = df['entry_published_date'].apply(lambda x: parser.parse(x).date()) #parse different date formats to date object format
dbdf = pd.read_csv('RSS_sports_feeds.csv')
dbdf['team_or_player'] = df['entry_title']
dbdf['source'] = df['feed_title']
dbdf['publication_date'] = df['publication_date'] 
dbdf['content'] = df['entry_summary']
dbdf['trust_score'] = 0.00  #default
dbdf['classification'] = 'unknown' #default
dbdf['link'] = df['entry_link']

#make a new df in the format of the DB table for easy inserting
sports_DB_df = dbdf[['team_or_player', 'source', 'publication_date', 'content', 'trust_score', 'classification', 'link']]
#save to a new CSV 
sports_DB_df.to_csv('formatted_sports_posts_for_DB.csv', index=False)
print("done")

done


C:\Users\tizia\Anaconda4\Lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


### Examine 2 Lists of True and False Headlines to Discover Features for Heuristic

In [9]:
#Define Lists
false_headlines=[
    "BREAKING: Fox Sports Cancels ALL NFL Broadcasts \"Until Players Respect The Flag\"",
    "Rugby Safer Than American Football \"For Health Reasons\": Biden",
    "First woman to medal in six straight Olympics. Media and sponsors ignore her because she is outspoken pro-2A.",
    "Novak Djokovic becomes the first professional athlete in history to be banned from a major sporting competition for not taking drugs",
    "BREAKING: ESPN has fired Shannon Sharpe, per @ESPNNBA",
    "The Minnesota Vikings have denounced Tim Walz: \"We don’t suppᴏrt his values.\"",
    "'BREAKING: WNBA referees disqualify two players under league’s new \"no anthem kneeling\" rule",
    "Nike announces termination of contract with Brittney Griner after \"strong backlash\" from online community: \"We need more athletes like Riley Gaines and less woke Brittney Griner!\"",
    "KNEELING: After the University of Texas, all students who knelt during the national anthem were rounded up and REMOVED FROM SCHOLARSHIPS",
    "Travis Kelce kneels during national anthem fined $10 million and thrown out of the game.",
    "The NFL will now use facial recognition at every stadium to verify the identity of everyone at the game.",
    "Mike Tyson says he’s willing to box Olympic DUDE with all proceeds to go to a battered women’s charity.",
    "After winning silver, Yusef stood emotionless on the Olympic podium and declared, \"Sharon, if you’re watching this, I want my dog back.\"",
    "Miami Dolphins QB Tua Tagovailoa will be sitting front row tonight in Doral for the Trump speech",
    "BREAKING: The WNBA organizers have officially announced an investigation into the referees in all of Caitlin Clark's games for ignoring all dirty actions by her opponents against her",
    "Chiefs' Coach Andy Reid \"fires 3 top players for anthem kneeling.\"",
    "BREAKING: Caitlin Clark Rejects $400 Million Deal From Nike, \"Not With That Kaepernick Clown,\"",
    "At Euro 2020, UEFA (European Football Association) ordered all team captains to wear \"OneLove\" bands. The band was used as a symbol of LGBTQ. But, Portugal captain Cristiano Ronaldo was the only European captain who did not wear the band.",
    "Golden State Warriors refuse to visit White House after winning NBA title: reports",
    "Taylor Swift faces a 10-game NFL ban following controversial political involvement - fans in uproar!"
]
true_headlines=[
    "Did that really happen? Barbados v Grenada and a deliberate own goal",
    "Breakdancing Will Not Be Busting A Move In 2028 Olympics",
    "Breaking Will Not Be in The 2028 Los Angeles Olympics—What’s Next?",
    "Braves Superstar Sets Atlanta-Era Record in 1st Inning",
    "Only 20 schools in the Football Bowl Subdivision have athletic departments with revenue exceeding expenses",
    "LATEST: Mitchell Stadium named America’s Best High School Football Stadium",
    "Homes of Patrick Mahomes, Travis Kelce burglarized last month",
    "Gregg Popovich recovering from mild stroke, no timeline for return set",
    "Patrick Queen: I wasn’t wanted back with Ravens, it was definitely kind of upsetting",
    "Odell Beckham Jr. boasts about taking 2021 Rams salary in Bitcoin National Football League Odell Beckham Jr. boasts about taking 2021 Rams salary in Bitcoin",
    "Ecuador international soccer player Marco Angulo dies aged 22 following car crash",
    "Shohei Ohtani Baseball Worth $4 Million Lands in Globally-Recognized Skyscraper",
    "Kobe Bryant is the only person to have won both an Olympic medal and an Oscar",
    "Bucks fan predicted Milwaukee-Phoenix NBA Finals all the way back in 2016",
    "Chiefs kicker Butker congratulates women graduates and says most are more excited about motherhood",
    "Kansas City Chiefs player faces backlash for graduation speech criticizing working women, calling Pride a \"deadly sin\"", 
]


In [11]:
#!pip install textstat
from transformers import pipeline
import string
from textstat import flesch_reading_ease
import textstat
import re


#functions for each feature we are considering
def word_count(text):
    return len(text.split())

def char_count(text):
    return len(text)

def avg_word_length(text):
    words = text.split()
    return sum(len(word) for word in words) / len(words) if words else 0

def all_capitalized_word_count(text):
    return sum(1 for word in text.split() if word.isupper())

def punctuation_count(text):
    return sum(1 for char in text if char in string.punctuation)

def exclamation_count(text):
    return text.count("!")

def question_mark_count(text):
    return text.count("?")

def readability_score(text): #high score = easier readability
    return flesch_reading_ease(text)

def smog_index(text): #readability based on the number of complex words (3+ syllables)
    return textstat.smog_index(text)

def count_quotation_pairs(text): 
    double_quotes = re.findall(r'"[^"]+"', text) #Finds all sections of text in double quotes
    return len(double_quotes)

def variety_of_vocabularity(text):
    words = text.split()
    unique_words = set(words)
    return len(unique_words) / len(words) if words else 0


sensational_words = ["shocking", "amazing", "unbelievable", "you won’t believe", "incredible", "stunning",\
                    "astounding", "breathtaking", "outstanding", "thrilling", "must see", "terrible", "awful",\
                     "fantastic", "horrible", "remarkable"]
def count_sensational_words(text):
    text = text.lower()
    return sum(1 for word in sensational_words if word in text)


absolute_words = ["always", "never", "clearly", "obviously", "definitely", "everyone", "nobody", "all",\
              "none", "blatantly", "undoubtedly", "must", "should"]
def count_absolute_words(text):
    words = text.lower().split()
    return sum(1 for word in words if word in absolute_words)


factuality_words = ["proven", "irrefutable", "unarguable", "unquestionably", "certainly", "definitely",\
                    "undeniable"]
def count_factuality_words(text):
    words = text.lower().split()
    return sum(1 for word in words if word in factuality_words)


subjective_words = ["believe", "think", "feel", "prefer", "seems", "wish"]
def subjectivity_score(text):
    words = text.lower().split()
    subjective_count = sum(1 for word in words if word in subjective_words)
    return subjective_count / len(words) if len(words) > 0 else 0

objective_words = ["reported", "measured", "confirmed", "analyzed", "observed", "recorded",\
                   "found", "documented", "verified", "tested", "studied", "calculated", "noted",\
                   "established", "evidence", "fact", "data", "statistics", "demonstrated", "shown",\
                   "results", "result", "evidence-based", "peer-reviewed", "sampled", "quantified",\
                   "evaluated", "experimented", "investigated"]
def objective_score(text):
    words = text.lower().split()
    objective_count = sum(1 for word in words if word in objective_words)
    return objective_count / len(words) if len(words) > 0 else 0


def count_numerics(text):
    numerical_count = len(re.findall(r'\d+', text)) 
    return numerical_count


pipe_finnews = pipeline("text-classification", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
def sentiment(text):
    result = pipe_finnews(text)
    return result[0]["label"] 
    
    
#add in source credibility 

In [ ]:
#Apply these feature functions to the headlines to compare
features = [
    word_count, char_count, avg_word_length, all_capitalized_word_count,
    punctuation_count, exclamation_count, question_mark_count, readability_score,
    smog_index, count_quotation_pairs, variety_of_vocabularity,
    count_sensational_words, count_absolute_words, count_factuality_words,
    subjectivity_score, objective_score, count_numerics, sentiment
]

def compute_features(headlines):
    results = []
    for headline in headlines:
        data = {
            "headline": headline,
            "word_count": word_count(headline),
            "char_count": char_count(headline),
            "avg_word_length": avg_word_length(headline),
            "all_capitalized_word_count": all_capitalized_word_count(headline),
            "punctuation_count": punctuation_count(headline),
            "exclamation_count": exclamation_count(headline),
            "question_mark_count": question_mark_count(headline),
            "readability_score": readability_score(headline),
            "smog_index": smog_index(headline),
            "quotation_count": count_quotation_pairs(headline),
            "vocab_variety": variety_of_vocabularity(headline),
            "sensational_words_count": count_sensational_words(headline),
            "absolute_words_count": count_absolute_words(headline),
            "factuality_words_count": count_factuality_words(headline),
            "subjectivity_score": subjectivity_score(headline),
            "objective_score": objective_score(headline),
            "count_numerics": count_numerics(headline),
            "sentiment": sentiment(headline)
        }
        results.append(data)
    return pd.DataFrame(results)

# Compute features for true and false headlines
true_features_df = compute_features(true_headlines)
false_features_df = compute_features(false_headlines)

true_features_df["label"] = "True"
false_features_df["label"] = "False"

combined_df = pd.concat([true_features_df, false_features_df], ignore_index=True)
combined_df.to_csv("headlines_initial_hueristic_exploration.csv")
print("Data saved to 'headlines_initial_hueristic_exploration.csv'")


### Run the Heuristics on the European Soccer Dataset

In [ ]:
real_euro_df = pd.read_csv('real_european.csv')
fake_euro_df = pd.read_csv('fake_european.csv')

real_euro_df["label"] = "True"
fake_euro_df["label"] = "False"

real_tweets= real_euro_df["tweet"].dropna()
fake_tweets= fake_euro_df["tweet"].dropna()

# Compute features for true and false tweets
euro_true_features_df = compute_features(real_tweets)
euro_false_features_df = compute_features(fake_tweets)

euro_true_features_df["label"] = "True"
euro_false_features_df["label"] = "False"

euro_combined_df = pd.concat([euro_true_features_df, euro_false_features_df], ignore_index=True)
euro_combined_df.to_csv("Euro_tweets_initial_hueristic_exploration.csv")
print("Data saved to 'Euro_tweets_initial_hueristic_exploration.csv'")

### Define labeling heuristic and trust score to update sports_DB_df with ground truths